## Notebook for deploying Cognite Function

### Authentication

In [1]:
from datetime import datetime
import pandas as pd
import os

from initialize import initialize_client

func_suffix = "Development"

In [2]:
ts_input_name = "VAL_11-LT-95034A:X.Value"
ts_output_name = "VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue"#f"VAL_11-LT-95034A:X.D.LeakValue"
tank_volume = 1400
derivative_value_excl = 0.002
# start_date = datetime(2023, 3, 21, 1, 0, 0)

data_dict = {'tot_days':0, 'tot_minutes':15, # convert date to str to make it JSON serializable
            'ts_input_name':ts_input_name, 'ts_output_name':ts_output_name,
            'derivative_value_excl':derivative_value_excl, 'tank_volume':tank_volume,
            'cdf_env':"dev", 'dataset_id': 1832663593546318} # NB: change dataset id when going to dev/test/prod!

cron_interval = "15"

In [3]:
client = initialize_client(data_dict["cdf_env"], cache_token=False)

In [28]:
myts = client.time_series.data.retrieve(external_id="pi:156793", limit=200).to_pandas()
pd.to_datetime(myts.index).date
# client.time_series.data.delete_range(start=datetime(2020,1,1), end=datetime(2023,10,10), external_id="VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue")

array([datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 2, 18),
       datetime.date(2020, 2, 18), datetime.date(2020, 

### Upload file to be associated with your dataset

In [5]:
folder = os.getcwd().replace("\\", "/")
# function_path = "handler.py"
function_path = "zip_handler.zip"

uploaded = client.files.upload(path=f"{folder}/{function_path}", name=function_path, data_set_id=data_dict["dataset_id"])

### Deploy Cognite function

In [6]:
# Create function
func_drainage = client.functions.create(
    name=f"avg-drainage-rate",
    external_id=f"avg-drainage-rate",
    # folder=".",
    file_id=uploaded.id,
)

In [7]:
func_drainage = client.functions.retrieve(external_id=f"avg-drainage-rate")
func_drainage.status

'Deploying'

In [8]:
# Single call to function
call_func_drainage = func_drainage.call(data=data_dict)

### Schedule Cognite function

In [9]:
# Schedule
func_drainage_schedule = client.functions.schedules.create(
    name=f"avg-drainage-rate-schedule-{func_suffix}",
    cron_expression=f"*/{cron_interval} * * * *", # every 15 min
    function_id=func_drainage.id,
    #client_credentials=client,
    description="Calculation scheduled every hour",
    data=data_dict
)

### NB: Delete function
Hear with DataOps: how do you know when there is a backfilling in CDF for PI time series - are there automatic checks for this, what is best practice?

In [19]:
client.functions.delete(id=func_drainage.id)

client.time_series.delete(external_id=f"VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue{func_suffix}")

## Experimental

In [ ]:
from cognite.client.data_classes import EventFilter
# All events linked to either primary asset (7681193400980395) or any of its children assets
evtest = c.events.list(asset_subtree_ids=7681193400980395, limit=1000).to_pandas()
evtest["asset_ids"] = evtest["asset_ids"].apply(lambda x: x[0])
evtest.groupby(["asset_ids"]).size()